# Handling Missing Data in PySpark

In the real world, most datasets you work with will be incomplete, which means you will have missing data. You have 2 basic options for filling in missing data (you will personally have to make the decision for what is the right approach):

1. Drop the missing data points (including the possbily the entire row)
2. Fill them in with some other value (like the average).

There are also two different types of missing data to be aware of:

1. null values represents "no value" or "nothing", it's not even an empty string or zero. It can be used to represent that nothing useful exists. 
2. NaN stands for "Not a Number", it's usually the result of a mathematical operation that doesn't make sense, e.g. 0.0/0.0 

Let's cover examples of each of these methods!

In [1]:
import findspark

findspark.init()

import pyspark  # only run after findspark.init()
from pyspark.sql import SparkSession

# May take awhile locally
spark = SparkSession.builder.appName("nulls").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


## Read in data for this Notebook

In [2]:
# Start by reading a basic csv dataset
# Let Spark know about the header and infer the Schema types!

#Some csv data
zomato = spark.read.csv('Datasets/zomato.csv', inferSchema=True, header=True)

## About this dataset

This dataset contains the aggregate rating of restaurant in Bengaluru India from Zomato. 

**Source:** https://www.kaggle.com/himanshupoddar/zomato-bangalore-restaurants

In [3]:
print(zomato.printSchema())

root
 |-- url: string (nullable = true)
 |-- address: string (nullable = true)
 |-- name: string (nullable = true)
 |-- online_order: string (nullable = true)
 |-- book_table: string (nullable = true)
 |-- rate: string (nullable = true)
 |-- votes: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- location: string (nullable = true)
 |-- rest_type: string (nullable = true)
 |-- dish_liked: string (nullable = true)
 |-- cuisines: string (nullable = true)
 |-- approx_cost(for two people): string (nullable = true)
 |-- reviews_list: string (nullable = true)
 |-- menu_item: string (nullable = true)
 |-- listed_in(type): string (nullable = true)
 |-- listed_in(city): string (nullable = true)

None


In [3]:
# Edit some var types
from pyspark.sql.types import *
from pyspark.sql.functions import *

df = zomato.withColumn("approx_cost(for two people)", zomato["approx_cost(for two people)"].cast(IntegerType())) \
    .withColumn("votes", zomato["votes"].cast(IntegerType()))
#QA
print(df.printSchema())

root
 |-- url: string (nullable = true)
 |-- address: string (nullable = true)
 |-- name: string (nullable = true)
 |-- online_order: string (nullable = true)
 |-- book_table: string (nullable = true)
 |-- rate: string (nullable = true)
 |-- votes: integer (nullable = true)
 |-- phone: string (nullable = true)
 |-- location: string (nullable = true)
 |-- rest_type: string (nullable = true)
 |-- dish_liked: string (nullable = true)
 |-- cuisines: string (nullable = true)
 |-- approx_cost(for two people): integer (nullable = true)
 |-- reviews_list: string (nullable = true)
 |-- menu_item: string (nullable = true)
 |-- listed_in(type): string (nullable = true)
 |-- listed_in(city): string (nullable = true)

None


In [5]:
df.limit(4).toPandas()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775.0,080 42297555,None,None,None,None,NaN,None,None,None,None
1,"+91 9743772233""",Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,NaN,('Rated 4.0','RATED\n You canÃ\x83Ã\x83Ã\x82Ã\x82Ã\x...,('Rated 5.0','RATED\n Overdelighted by the service and fo...,('Rated 4.0',NaN,('Rated 4.0','RATED\n The place is nice and comfortable. ...,('Rated 4.0','RATED\n The place is nice and comfortable. ...
2,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787.0,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,"""[('Rated 4.0', 'RATED\n Had been here for di...",rice was well cooked and overall was great\n\n...,('Rated 5.0','RATED\n This place just cool ? with good am...
3,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918.0,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,"""[('Rated 3.0', """"RATED\n Ambience is not tha...",('Rated 3.0',"""""RATED\n \nWent there for a quick bite with ...",pasta churros and lasagne.\n\nNachos were pat...


Note that nulls values appear as "None" in the Pandas print out above. If we show the null values for the cuisines variable in attempt to view that first restaurant "Jalsa", we can see it appear as "null" below.

In [5]:
from pyspark.sql import functions as F

# zomato.filter("cuisines='None'").agg(F.count(zomato.name)).show()
df.filter(df.cuisines.isNull()).select(['name', 'cuisines']).limit(5).toPandas()

,name,cuisines
0,Jalsa,None
1,Grand Village,None
2,Casual Dining,None
3,Timepass Dinner,None
4,Casual Dining,None


## Missing Data Statistics

It is always valualuable to know how much missing data you are going to be working with before you take any action like filling missing values with an average or dropping rows completly. Here is a good script to get you started. We will also explore more later on in this notebook.

In [9]:
from pyspark.sql.functions import *


def null_value_calc(df):
    null_columns_counts = []
    numRows = df.count()
    for k in df.columns:
        nullRows = df.where(col(k).isNull()).count()
        if nullRows > 0:
            temp = k, nullRows, (nullRows / numRows) * 100
            null_columns_counts.append(temp)
    return null_columns_counts

null_columns_calc_list = null_value_calc(df)
spark.createDataFrame(null_columns_calc_list, ['Column_Name', 'Null_Values_Count', 'Null_Value_Percent']).toPandas()

,Column_Name,Null_Values_Count,Null_Value_Percent
0,name,85,0.118500
1,online_order,8111,11.307682
2,book_table,2,0.002788
3,rate,7775,10.839258
4,votes,20018,27.907431
5,phone,1227,1.710581
6,location,20054,27.957619
7,rest_type,20165,28.112366
8,dish_liked,46841,65.301826
9,cuisines,27305,38.066360


,Column_Name,Null_Values_Count,Null_Value_Percent
0,name,85,0.118500
1,online_order,8111,11.307682
2,book_table,2,0.002788
3,rate,7775,10.839258
4,votes,20018,27.907431
5,phone,1227,1.710581
6,location,20054,27.957619
7,rest_type,20165,28.112366
8,dish_liked,46841,65.301826
9,cuisines,27305,38.066360


In [10]:
# Another way if you prefer
from pyspark.sql.functions import isnan, when, count, col

#first row: null count
nulls = df.select(
    [count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
)
# Second row: null percent
percent = df.select(
    [format_number(
        ((count(when(isnan(c) | col(c).isNull(), c)) / df.count()) * 100), 1).alias(c) for c in df.columns])

result = nulls.union(percent)

result.toPandas()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,0,0,85,8111,2,7775,20018,1227,20054,20165,46841,27305,43611,28185,28611,28983,29344
1,0.0,0.0,0.1,11.3,0.0,10.8,27.9,1.7,28.0,28.1,65.3,38.1,60.8,39.3,39.9,40.4,40.9


## Drop the missing data

PySpark has a really handy .na function for working with missing data. The drop command has the following parameters:

    df.na.drop(how='any', thresh=None, subset=None)

In [11]:
# Drop any row that contains missing data across the whole dataset
df.na.drop().limit(4).toPandas()

# Note this statement is equivilant to the above:
# df.na.drop(how='any').limit(4).toPandas() 

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"""[('Rated 4.0', 'RATED\n Had been here for di...",rice was well cooked and overall was great\n\n...,('Rated 5.0','RATED\n This place just cool ? with good am...
1,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"""[('Rated 3.0', """"RATED\n Ambience is not tha...",('Rated 3.0',"""""RATED\n \nWent there for a quick bite with ...",pasta churros and lasagne.\n\nNachos were pat...
2,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"""[('Rated 4.0', """"RATED\n Great food and prop...",('Rated 2.0','RATED\n Reached the place at 3pm on Saturda...,('Rated 4.0'
3,https://www.zomato.com/bangalore/cafe-shuffle-...,"941, 3rd FLOOR, 21st Main, 22nd Cross, Banasha...",Cafe Shuffle,Yes,Yes,4.2/5,150,+91 9742166777,Banashankari,Cafe,"Mocktails, Peri Fries, Lasagne, Pizza, Chicken...","Cafe, Italian, Continental",600,"""[('Rated 1.0', """"RATED\n \n\nHorrible. Not ev...","you get it literally half an hour late."""")",('Rated 4.0',"""""RATED\n While this place is more common fo..."


In [12]:
# Of course you will want to know how many rows that affected before you actually execute it..
og_len = df.count()
drop_len = df.na.drop().count()
print("Total Rows Dropped:", og_len - drop_len)
print("Percentage of Rows Dropped", (og_len - drop_len) / og_len)

Total Rows Dropped: 63124
Percentage of Rows Dropped 0.8800223058692318


Woah! 88% is a lot! We better figure out a better method.

In [13]:
# Drop rows that have at least 8 NON-null values
og_len = df.count()
drop_len = df.na.drop(thresh=8).count()
print("Total Rows Dropped:", og_len - drop_len)
print("Percentage of Rows Dropped", (og_len - drop_len) / og_len)

Total Rows Dropped: 1694
Percentage of Rows Dropped 0.023616339049212325


Way better!

In [14]:
# Only drop the rows whose values in the sales column are null
og_len = df.count()
drop_len = df.na.drop(subset=["votes"]).count()
print("Total Rows Dropped:", og_len - drop_len)
print("Percentage of Rows Dropped", (og_len - drop_len) / og_len)

Total Rows Dropped: 20018
Percentage of Rows Dropped 0.2790743064268786


In [15]:
# Another way to do the above
og_len = df.count()
drop_len = df.filter(df.rate.isNotNull()).count()
print("Total Rows Dropped:", og_len - drop_len)
print("Percentage of Rows Dropped", (og_len - drop_len) / og_len)

Total Rows Dropped: 7775
Percentage of Rows Dropped 0.10839258329848041


In [16]:
# Drop a row only if ALL its values are null.
og_len = df.count()
drop_len = df.na.drop(how='all').count()
print("Total Rows Dropped:", og_len - drop_len)
print("Percentage of Rows Dropped", (og_len - drop_len) / og_len)

Total Rows Dropped: 0
Percentage of Rows Dropped 0.0


## Fill the missing values

We can also fill the missing values with new values. If you have multiple nulls across multiple data types, Spark is actually smart enough to match up the data types. For example:

In [17]:
# Fill all nulls values with one common value (character value)
df.na.fill('MISSING').limit(4).toPandas()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775.0,080 42297555,MISSING,MISSING,MISSING,MISSING,NaN,MISSING,MISSING,MISSING,MISSING
1,"+91 9743772233""",Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,NaN,('Rated 4.0','RATED\n You canÃ\x83Ã\x83Ã\x82Ã\x82Ã\x...,('Rated 5.0','RATED\n Overdelighted by the service and fo...,('Rated 4.0',NaN,('Rated 4.0','RATED\n The place is nice and comfortable. ...,('Rated 4.0','RATED\n The place is nice and comfortable. ...
2,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787.0,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,"""[('Rated 4.0', 'RATED\n Had been here for di...",rice was well cooked and overall was great\n\n...,('Rated 5.0','RATED\n This place just cool ? with good am...
3,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918.0,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,"""[('Rated 3.0', """"RATED\n Ambience is not tha...",('Rated 3.0',"""""RATED\n \nWent there for a quick bite with ...",pasta churros and lasagne.\n\nNachos were pat...


In [18]:
# Fill all nulls values with one common value (numeric value)
df.na.fill(999).limit(10).toPandas()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555,None,None,None,None,999,None,None,None,None
1,"+91 9743772233""",Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,999,('Rated 4.0','RATED\n You canÃ\x83Ã\x83Ã\x82Ã\x82Ã\x...,('Rated 5.0','RATED\n Overdelighted by the service and fo...,('Rated 4.0',999,('Rated 4.0','RATED\n The place is nice and comfortable. ...,('Rated 4.0','RATED\n The place is nice and comfortable. ...
2,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"""[('Rated 4.0', 'RATED\n Had been here for di...",rice was well cooked and overall was great\n\n...,('Rated 5.0','RATED\n This place just cool ? with good am...
3,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"""[('Rated 3.0', """"RATED\n Ambience is not tha...",('Rated 3.0',"""""RATED\n \nWent there for a quick bite with ...",pasta churros and lasagne.\n\nNachos were pat...
4,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"""[('Rated 4.0', """"RATED\n Great food and prop...",('Rated 2.0','RATED\n Reached the place at 3pm on Saturda...,('Rated 4.0'
5,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447,None,None,None,None,999,None,None,None,None
6,"+91 9901210005""",Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,999,[],Buffet,Banashankari,None,None,999,None,None,None,None
7,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,Yes,No,3.8/5,286,+91 9980040002,None,None,None,None,999,None,None,None,None
8,"+91 9980063005""",Basavanagudi,Casual Dining,"Onion Rings, Pasta, Kadhai Paneer, Salads, Sal...",North Indian,600,999,[],Buffet,Banashankari,None,None,999,None,None,None,None
9,https://www.zomato.com/bangalore/rosewood-inte...,"19/1, New Timberyard Layout, Beside Satellite ...",Rosewood International Hotel - Bar & Restaurant,No,No,3.6/5,8,+91 9731716688,None,None,None,None,999,None,None,None,None


Usually you should specify what columns you want to fill with the subset parameter

In [19]:
df.filter(df.name.isNull()).na.fill('No Name', subset=['name']).limit(5).toPandas()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,"+91 9986692090""",BTM,No Name,"Momos, Oreo Shake","Mughlai, North Indian, Chinese, Momos",600,NaN,('Rated 3.0','RATED\n Simple food with great north indian...,"['Fry Chicken Kabab [5 Pieces]', 'Fry Chicken ...",Delivery,Bannerghatta Road,NaN,None,None,None,None
1,"00 805074123""",BTM,No Name,None,"North Indian, Chinese, Arabian",700,NaN,[],Delivery,Bannerghatta Road,None,None,NaN,None,None,None,None
2,"+91 8971051846""",Bannerghatta Road,No Name,None,"Street Food, Burger",150,NaN,[],Delivery,Bannerghatta Road,None,None,NaN,None,None,None,None
3,"080 39457777""",Bannerghatta Road,No Name,"Chicken Biryani, Hyderabadi Biryani, Rolls, Mu...","Biryani, North Indian",500,NaN,('Rated 3.0','RATED\n too much oil in rice items'),('Rated 2.0','RATED\n salan was not provided'),('Rated 1.0',NaN,('Rated 3.0','RATED\n ok ok biryani'),('Rated 1.0',"""""RATED\n poor test & quality... will never ..."
4,"+91 8971051846""",Bannerghatta Road,No Name,None,"Street Food, Burger",150,NaN,[],Dine-out,Bannerghatta Road,None,None,NaN,None,None,None,None


A very common practice is to fill values with the **mean value** for the column. Here is a fun function to that in an automatted way.

In [25]:
def fill_with_mean(df, include=None):
    if include is None:
        include = set()
    stats = (
        df.agg(
            *(avg(c).alias(c) for c in df.columns if c in include))
    )
    print(stats.first().asDict())
    return df.na.fill(stats.first().asDict())


updated_df = fill_with_mean(df, ["votes", "approx_cost(for two people)"])
updated_df.toPandas()

{'votes': 283.71422493811883, 'approx_cost(for two people)': 387.40837156371134}


,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555,None,None,None,None,387,None,None,None,None
1,"+91 9743772233""",Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,283,('Rated 4.0','RATED\n You canÃ\x83Ã\x83Ã\x82Ã\x82Ã\x...,('Rated 5.0','RATED\n Overdelighted by the service and fo...,('Rated 4.0',387,('Rated 4.0','RATED\n The place is nice and comfortable. ...,('Rated 4.0','RATED\n The place is nice and comfortable. ...
2,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"""[('Rated 4.0', 'RATED\n Had been here for di...",rice was well cooked and overall was great\n\n...,('Rated 5.0','RATED\n This place just cool ? with good am...
3,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"""[('Rated 3.0', """"RATED\n Ambience is not tha...",('Rated 3.0',"""""RATED\n \nWent there for a quick bite with ...",pasta churros and lasagne.\n\nNachos were pat...
4,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"""[('Rated 4.0', """"RATED\n Great food and prop...",('Rated 2.0','RATED\n Reached the place at 3pm on Saturda...,('Rated 4.0'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71725,https://www.zomato.com/bangalore/best-brews-fo...,"Four Points by Sheraton Bengaluru, 43/3, White...",Best Brews - Four Points by Sheraton Bengaluru...,No,No,3.6 /5,27,080 40301477,Whitefield,Bar,None,Continental,387,"""[('Rated 5.0', """"RATED\n Food and service ar...",polite,and always willing to go the extra mile to he...,('Rated 3.5'
71726,https://www.zomato.com/bangalore/vinod-bar-and...,"Number 10, Garudachar Palya, Mahadevapura, Whi...",Vinod Bar And Restaurant,No,No,None,0,+91 8197675843,Whitefield,Bar,None,Finger Food,600,[],[],Pubs and bars,Whitefield
71727,https://www.zomato.com/bangalore/plunge-sherat...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Plunge - Sheraton Grand Bengaluru Whitefield H...,No,No,None,0,None,Whitefield,Bar,None,Finger Food,387,[],[],Pubs and bars,Whitefield
71728,https://www.zomato.com/bangalore/chime-sherato...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,Yes,4.3 /5,236,080 49652769,"ITPL Main Road, Whitefield",Bar,"Cocktails, Pizza, Buttermilk",Finger Food,387,"""[('Rated 4.0', 'RATED\n Nice and friendly pl...",('Rated 5.0','RATED\n I visited chime bar at Sheraton hot...,('Rated 4.0'


## Keeping the missing data
A few machine learning algorithms can easily deal with missing data. Just do your research and make sure the nulls values are not impacting the integrity of your analysis. 

That is all we need to know for now!